**Задание**
Берем отызывы за лето (из архива с материалами или предыдущего занятия)

1. Учим conv сеть для классификации

2. Рассмотреть 2-а варианта сеточек 

2.1 Инициализировать tf.keras.layers.Embedding предобученными векторами взять к примеру с https://rusvectores.org/ru/

2.2 Инициализировать слой tf.keras.layers.Embedding по умолчанию (ну то есть вам ничего не делать с весами)

 				
Сравнить две архитектуры с предобученными весами и когда tf.keras.layers.Embedding обучается сразу со всей сеточкой, что получилось лучше

In [15]:
import pandas as pd
from sklearn.model_selection import train_test_split
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re

In [186]:
max_words = 10000
max_len = 40
num_classes = 1
EMBEDDING_DIM = 300
# Training
epochs = 10
batch_size = 512
print_batch_n = 100

In [53]:
data = pd.read_excel('data.xls')

In [54]:
data = data[(data['Rating'] != 3)]

In [55]:
data['Rating'] = (data['Rating'] > 3).astype(int)

In [56]:
X_train, X_test, y_train, y_test = train_test_split(data['Content'], data['Rating'], test_size=0.33, random_state=42)

#### Подготовим текст, уберем стопслова, пунктуацию, приведем слова к нормальной форме.

In [57]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)


In [58]:
X_train = X_train.apply(preprocess_text)

In [59]:
X_test = X_test.apply(preprocess_text)

#### Соберем весть трейн сет в корпус слов (склеим все строки) и разобьем на токены.

In [60]:
train_corpus = " ".join(X_train)
train_corpus = train_corpus.lower()

In [61]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download("punkt")

tokens = word_tokenize(train_corpus)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


#### Отфильтруем данные и соберём в корпус N наиболее частых токенов (параметр max_world)

In [62]:
tokens_filtered = [word for word in tokens if word.isalnum()]  # Фильтруем, чтобы не было чисел в токенах, т.к. они бесполезны

In [63]:
from nltk.probability import FreqDist
dist = FreqDist(tokens_filtered)
tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]

In [64]:
tokens_filtered_top[:10]

['приложение',
 'удобно',
 'работать',
 'удобный',
 'отлично',
 'нравиться',
 'отличный',
 'хороший',
 'супер',
 'телефон']

In [65]:
vocabulary = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()} # Делаем словарь типа слово:индекс слова

#### Переводим текст в массив нампи, где значение ячейки = индексу слова. Длина строки - параметр max_len

In [66]:
import numpy as np

def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])
    padding = [0]*(maxlen-len(result))
    return padding + result[-maxlen:]

In [67]:
X_train = np.asarray([text_to_sequence(text, max_len) for text in X_train], dtype=np.int32)
X_test = np.asarray([text_to_sequence(text, max_len) for text in X_test], dtype=np.int32)

In [68]:
X_train[4]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,  308,    1, 2206])

#### Создаем модель на Keras

In [226]:
import numpy as np
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.callbacks import TensorBoard 
from keras.losses import BinaryCrossentropy 
from keras.callbacks import EarlyStopping  

In [227]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=EMBEDDING_DIM, input_length=max_len))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('sigmoid'))

In [228]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [229]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/10
24/24 [==============================] - 1s 31ms/step - loss: 0.5097 - accuracy: 0.8221 - val_loss: 0.3815 - val_accuracy: 0.8565
Epoch 2/10
24/24 [==============================] - 0s 10ms/step - loss: 0.3241 - accuracy: 0.8670 - val_loss: 0.2390 - val_accuracy: 0.9041


In [230]:
results = model.predict(X_test, batch_size=batch_size, verbose=1)

13/13 [==============================] - 0s 2ms/step


In [231]:
from sklearn.metrics import f1_score, classification_report, roc_auc_score

In [232]:
results_tresh = (results > 0.5) * 1

In [233]:
print(classification_report(y_test, results_tresh))

              precision    recall  f1-score   support

           0       0.78      0.51      0.62      1029
           1       0.91      0.97      0.94      5488

    accuracy                           0.90      6517
   macro avg       0.84      0.74      0.78      6517
weighted avg       0.89      0.90      0.89      6517



In [234]:
print(roc_auc_score(y_test, results))

0.945660396603456


#### Вторая модель. 

In [235]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=EMBEDDING_DIM, input_length=max_len))
model.add(Conv1D(256, 2))
model.add(Activation("relu"))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(Conv1D(128, 4))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(100))
model.add(Activation("relu"))
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('sigmoid'))

In [236]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [237]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/10
24/24 [==============================] - 1s 23ms/step - loss: 0.4720 - accuracy: 0.8187 - val_loss: 0.3197 - val_accuracy: 0.8603
Epoch 2/10
24/24 [==============================] - 0s 14ms/step - loss: 0.2488 - accuracy: 0.8960 - val_loss: 0.2128 - val_accuracy: 0.9086


In [238]:
results = model.predict(X_test, batch_size=batch_size, verbose=1)

13/13 [==============================] - 0s 3ms/step


In [239]:
from sklearn.metrics import f1_score, classification_report, roc_auc_score

In [240]:
results_tresh = (results > 0.5) * 1

In [241]:
print(classification_report(y_test, results_tresh))

              precision    recall  f1-score   support

           0       0.73      0.73      0.73      1029
           1       0.95      0.95      0.95      5488

    accuracy                           0.91      6517
   macro avg       0.84      0.84      0.84      6517
weighted avg       0.91      0.91      0.91      6517



In [242]:
print(roc_auc_score(y_test, results))

0.9382272692500574


#### Возьмем предобученный слой ембедингов

model.txt это скаченная с https://rusvectores.org/ru/ матрица эмбедингов на русские слова. Модель обучалась с дополнительными метками частей речи, но т.к. мы изначально их не использовали, то и отсюда удалим, чтоб мы могли использовать наш предобработанный текст.

In [243]:
import re

In [244]:
embeddings_index = {}
with open('model.txt', encoding="utf-8") as f:
    for line in f:
        values = line.split()
        try:
            word = re.findall(r'(\w+)_\w+', values[0])[0]
        except:
            word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
        
            

print('Found %s word vectors.' % len(embeddings_index))

Found 157543 word vectors.


#### Мы слоставили словарь типа слово:вектор эмбединга из файла модели. Теперь мы берем наш словарь который мы составили на основе наших текстов, и создаем матрицу весов. Т.е. берем слово в нашем словаре и по его индексу в словаре вставляем вектор эмбединга.

In [245]:
embedding_matrix = np.zeros((max_words, EMBEDDING_DIM))
for word, i in vocabulary.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [246]:
len(embedding_matrix)

10000

#### Теперь создаем сетку, и в качестве матрицы весов эмбединга задаем подготовленную матрицу. В первом варианте мы не будем дообучать наш эмбединг, во втором будем дообучать.

In [247]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=EMBEDDING_DIM, weights=[embedding_matrix], input_length=max_len, trainable=False))
model.add(Conv1D(256, 2))
model.add(Activation("relu"))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(Conv1D(128, 4))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(100))
model.add(Activation("relu"))
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('sigmoid'))

In [248]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [249]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/10
24/24 [==============================] - 2s 25ms/step - loss: 0.3921 - accuracy: 0.8608 - val_loss: 0.2709 - val_accuracy: 0.8829
Epoch 2/10
24/24 [==============================] - 0s 16ms/step - loss: 0.2318 - accuracy: 0.8996 - val_loss: 0.2246 - val_accuracy: 0.9011


In [250]:
results = model.predict(X_test, batch_size=batch_size, verbose=1)

13/13 [==============================] - 0s 5ms/step


In [251]:
from sklearn.metrics import f1_score, classification_report, roc_auc_score

In [252]:
results_tresh = (results > 0.5) * 1

In [253]:
print(classification_report(y_test, results_tresh))

              precision    recall  f1-score   support

           0       0.68      0.71      0.70      1029
           1       0.95      0.94      0.94      5488

    accuracy                           0.90      6517
   macro avg       0.81      0.82      0.82      6517
weighted avg       0.90      0.90      0.90      6517



In [254]:
print(roc_auc_score(y_test, results))

0.9386883866416204


#### Второй вариант

In [255]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=EMBEDDING_DIM, weights=[embedding_matrix], input_length=max_len))
model.add(Conv1D(256, 2))
model.add(Activation("relu"))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(Conv1D(128, 4))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(100))
model.add(Activation("relu"))
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('sigmoid'))

In [256]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [257]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/10
24/24 [==============================] - 1s 27ms/step - loss: 0.3898 - accuracy: 0.8446 - val_loss: 0.2706 - val_accuracy: 0.8557
Epoch 2/10
24/24 [==============================] - 0s 20ms/step - loss: 0.2346 - accuracy: 0.8489 - val_loss: 0.2155 - val_accuracy: 0.8625


In [258]:
results = model.predict(X_test, batch_size=batch_size, verbose=1)

13/13 [==============================] - 0s 4ms/step


In [259]:
from sklearn.metrics import f1_score, classification_report, roc_auc_score

In [260]:
results_tresh = (results > 0.5) * 1

In [261]:
print(classification_report(y_test, results_tresh))

              precision    recall  f1-score   support

           0       0.64      0.21      0.32      1029
           1       0.87      0.98      0.92      5488

    accuracy                           0.86      6517
   macro avg       0.76      0.59      0.62      6517
weighted avg       0.83      0.86      0.83      6517



In [262]:
print(roc_auc_score(y_test, results))

0.9381632546812978


#### В Целом если не дообучать эмбединг, то лучше результат на f1 score, да и вообще лучший результат. Если включить дообучение, то ввиду дисбаланса классов, модель работает хуже.